# Filtros lineales espaciales - otros casos

<div class="alert alert-block alert-success">
<b>Resumen:</b> En esta actividad se aplican diversos filtros lineales espaciales a una imagen en escala de grises. Se crean y visualizan máscaras de filtros como el Laplaciano, promedio, Gaussiano, Laplaciano del Gaussiano (LoG), desenfoque de movimiento, Prewitt, Sobel y Unsharp Mask. Estos filtros se aplican a la imagen original para resaltar diferentes características como bordes, texturas y detalles. Se identifican las imágenes que requieren normalización debido a valores de píxeles fuera del rango estándar, y se aplica la normalización min-max para escalar los valores entre 0 y 255, permitiendo una visualización adecuada. Finalmente, se analizan los resultados para comprender el impacto de cada filtro y la importancia de la normalización en el procesamiento y visualización de imágenes digitales.
</div>

***

### 1. Preámbulo

In [ ]:
# Importar las bibliotecas necesarias
import cv2
import numpy as np
import matplotlib.pyplot as plt

### 2. Lectura de la imagen

In [ ]:
# Lectura y visualización de la imagen
im_gray_1 = cv2.imread('./images/moon.tif', cv2.IMREAD_GRAYSCALE)

# Mostrar la imagen original
plt.figure(figsize=(6,6))
plt.imshow(im_gray_1, cmap='gray')
plt.title('Imagen Original')
plt.axis('off')
plt.show()

### 3. Filtros

Creación de máscaras

In [ ]:
sz = (21, 21)

#### 3.1 Filtro Laplaciano

In [ ]:
w_1 = np.array([[0, 1, 0],
                [1, -4, 1],
                [0, 1, 0]], dtype=np.float32)

#### 3.2 Filtro Promedio (average)

In [ ]:
w_2 = np.ones(sz, dtype=np.float32) / (sz[0] * sz[1])

#### 3.3 Filtro Gaussiano

In [ ]:
sigma_gauss = 2.5
w_3 = cv2.getGaussianKernel(ksize=sz[0], sigma=sigma_gauss)
w_3 = w_3 * w_3.T  # Convertir a 2D

#### 3.4 Filtro Laplaciano del Gaussiano (LoG)

In [ ]:
sigma_log = 5
x, y = np.meshgrid(np.arange(-sz[0]//2+1, sz[1]//2+1), np.arange(-sz[0]//2+1, sz[1]//2+1))
w_4 = -(1/(np.pi * sigma_log**4)) * (1 - ((x**2 + y**2) / (2 * sigma_log**2))) * np.exp(-(x**2 + y**2) / (2 * sigma_log**2))
w_4 = w_4 / np.sum(np.abs(w_4))

#### 3.5 Filtro de movimiento (Motion Blur)

In [ ]:
size = 20
angle = 45
w_5 = np.zeros((size, size), dtype=np.float32)
center = size // 2
slope = np.tan(np.deg2rad(angle))
for i in range(size):
    offset = int(round(slope * (i - center) + center))
    if 0 <= offset < size:
        w_5[offset, i] = 1
w_5 /= w_5.sum()

#### 3.6 Filtro Prewitt

In [ ]:
w_6 = np.array([[-1, 0, 1],
                [-1, 0, 1],
                [-1, 0, 1]], dtype=np.float32)

#### 3.7 Filtro Sobel

In [ ]:
w_7 = np.array([[-1, 0, 1],
                [-2, 0, 2],
                [-1, 0, 1]], dtype=np.float32)

#### 3.8 Filtro Unsharp mask

In [ ]:
alpha = 0.2
kernel_size = 5
gaussian = cv2.getGaussianKernel(kernel_size, 0)
gaussian = gaussian * gaussian.T
delta = np.zeros_like(gaussian)
delta[kernel_size//2, kernel_size//2] = 1
w_8 = delta + alpha * (delta - gaussian)

#### 3.9 Visualización de la máscaras

In [ ]:
# Visualización de todas las máscaras en 2D

# Lista de máscaras y títulos
mascaras = [
    (w_1, 'Máscara Laplaciana (w_1)'),
    (w_2, 'Máscara Promedio (w_2)'),
    (w_3, 'Máscara Gaussiana (w_3)'),
    (w_4, 'Máscara LoG (w_4)'),
    (w_5, 'Máscara Motion Blur (w_5)'),
    (w_6, 'Máscara Prewitt (w_6)'),
    (w_7, 'Máscara Sobel (w_7)'),
    (w_8, 'Máscara Unsharp Mask (w_8)')
]

# Visualizar cada máscara
for mask, title in mascaras:
    plt.figure(figsize=(3,3))
    plt.imshow(mask, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.colorbar()
    plt.show()

#### 4. Filtrado

In [ ]:
def min_max_normalization(image, new_min=0, new_max=1):
    # Convertir a float para evitar desbordamientos
    image_float = image.astype(np.float32)
    norm_image = (image_float - image_float.min()) / (image_float.max() - image_float.min())
    norm_image = norm_image * (new_max - new_min) + new_min
    return norm_image

In [ ]:
# Convertir la imagen a tipo float32 para evitar problemas en operaciones
im_float = im_gray_1.astype(np.float32)

# Aplicar los filtros con replicación en los bordes
g_1 = cv2.filter2D(im_float, -1, w_1, borderType=cv2.BORDER_REPLICATE)    # Laplacian
g_2 = cv2.filter2D(im_float, -1, w_2, borderType=cv2.BORDER_REPLICATE)    # Average
g_3 = cv2.filter2D(im_float, -1, w_3, borderType=cv2.BORDER_REPLICATE)    # Gaussian
g_4 = cv2.filter2D(im_float, -1, w_4, borderType=cv2.BORDER_REPLICATE)    # LoG
g_5 = cv2.filter2D(im_float, -1, w_5, borderType=cv2.BORDER_REPLICATE)    # Motion Blur
g_6 = cv2.filter2D(im_float, -1, w_6, borderType=cv2.BORDER_REPLICATE)    # Prewitt
g_7 = cv2.filter2D(im_float, -1, w_7, borderType=cv2.BORDER_REPLICATE)    # Sobel
g_8 = cv2.filter2D(im_float, -1, w_8, borderType=cv2.BORDER_REPLICATE)    # Unsharp Mask

<b>Actividad</b>
- Identifique las imágenes que requieren normalización.
- Aplique la normalización min-max.

#### 5. Visualización de resultados

In [ ]:
# Lista de resultados y títulos
resultados = [
    (im_gray_1, 'Imagen Original'),
    (g_1, 'g_1 (Laplacian)'),
    (g_2, 'g_2 (Average)'),
    (g_3, 'g_3 (Gaussian)'),
    (g_4, 'g_4 (LoG)'),
    (g_5, 'g_5 (Motion Blur)'),
    (g_6, 'g_6 (Prewitt)'),
    (g_7, 'g_7 (Sobel)'),
    (g_8, 'g_8 (Unsharp Mask)')
]

# Visualizar cada resultado
for img, title in resultados:
    plt.figure(figsize=(6,6))
    plt.imshow(img, cmap='gray')
    plt.title(title)
    plt.axis('off')
    plt.show()

#### 6. Bonus track

In [ ]:
# Restar el filtro Laplaciano de la imagen original
g_9 = min_max_normalization(im_float - g_1)

plt.figure(figsize=(6,6))
plt.imshow(g_9, cmap='gray')
plt.title('g_9 (Imagen original - Laplacian)')
plt.axis('off')
plt.show()

***

<b> Actividad </b>
Implementar el LoG siguiendo https://docs.opencv.org/3.4/d5/db5/tutorial_laplace_operator.html

In [ ]:
# aquí su código